In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/summarization/cnn-news/translated-cnn-test.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/summarization/dailymail/translated-dailymail-test.json

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
import tokenization

tokenizer = tokenization.FullTokenizer(
    vocab_file='pegasus.wordpiece', do_lower_case=False
)

In [4]:
import tensorflow as tf

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


In [5]:
g = load_graph('output/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
top_p = g.get_tensor_by_name('import/top_p:0')
temperature = g.get_tensor_by_name('import/temperature:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [6]:
import json

with open('translated-cnn-test.json') as fopen:
    data = json.load(fopen)
    
with open('translated-dailymail-test.json') as fopen:
    data.extend(json.load(fopen))
    
len(data)

12000

In [7]:
X, Y = [], []
for i in range(len(data)):
    X.append(' '.join(data[i]['ms_article']))
    Y.append(' '.join(data[i]['ms_abstract']))

In [8]:
import re
from unidecode import unidecode
from malaya.text.rules import normalized_chars

def filter_news(string):
    string = string.lower()
    return 'javascript is disabled' in string or 'requires javascript' in string or 'javascript' in string \
    or 'président' in string

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def transformer_textcleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [9]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [10]:
from tqdm import tqdm

batch_size = 10
results = []
for i in tqdm(range(0, len(X), batch_size)):
    batch_x = X[i: i + batch_size]
    batches = []
    for b in batch_x:
        encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(f'{transformer_textcleaning(b)}'))
        encoded = encoded[:512] 
        batches.append(encoded)
    batches = pad_sequences(batches, padding='post')
    g = test_sess.run(logits, feed_dict = {x:batches, top_p: 0.0, temperature: 0.0})
    results.extend(g.tolist())

100%|██████████| 1200/1200 [1:14:36<00:00,  3.73s/it]


In [11]:
from tensor2tensor.utils import rouge
from tensorflow.keras.preprocessing import sequence

In [12]:
import numpy as np

In [13]:
def calculate_rouges(predicted, batch_y, n_size = 2):
    batch_y = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(f'{transformer_textcleaning(row)}')) for row in batch_y]
    maxlen = max(max(len(row) for row in batch_y), max(len(row) for row in predicted))
    batch_y = sequence.pad_sequences(batch_y, padding = 'post', maxlen = maxlen)
    predicted = sequence.pad_sequences(predicted, padding = 'post', maxlen = maxlen)
    
    non = np.count_nonzero(batch_y, axis = 1)
    o = []
    for n in non:
        o.append([True for _ in range(n)])
    b = sequence.pad_sequences(o, dtype = np.bool, padding = 'post', value = False, maxlen = maxlen)
    rouges = []
    for i in range(b.shape[0]):
        a = batch_y[i][b[i]]
        p = predicted[i][b[i]]
        score = rouge.rouge_n([a], [p], n = n_size)
        rouges.append(score)
    return np.mean(rouges)

def calculate_rouge_l(predicted, batch_y):
    batch_y = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(f'{transformer_textcleaning(row)}')) for row in batch_y]
    maxlen = max(max(len(row) for row in batch_y), max(len(row) for row in predicted))
    batch_y = sequence.pad_sequences(batch_y, padding = 'post', maxlen = maxlen)
    predicted = sequence.pad_sequences(predicted, padding = 'post', maxlen = maxlen)
    
    non = np.count_nonzero(batch_y, axis = 1)
    o = []
    for n in non:
        o.append([True for _ in range(n)])
    b = sequence.pad_sequences(o, dtype = np.bool, padding = 'post', value = False, maxlen = maxlen)
    rouges = []
    for i in range(b.shape[0]):
        a = batch_y[i][b[i]]
        p = predicted[i][b[i]]
        score = rouge.rouge_l_sentence_level([a], [p])
        rouges.append(score)
    return np.mean(rouges)

In [14]:
calculate_rouges(results, Y[:len(results)], n_size = 1)

0.29012334

In [15]:
calculate_rouges(results, Y[:len(results)], n_size = 2)

0.11878814

In [16]:
calculate_rouge_l(results, Y[:len(results)])

0.19232224